In [83]:
import os
import pandas as pd
import numpy as np

from utils.datatools import *

In [84]:
PHASEPATH = "/Users/sujinlee/desktop/git_repositories/nims_pd/dataset_LHEE_RHEE_Z_PHASE/"
SAVEPATH = "/Users/sujinlee/desktop/git_repositories/nims_pd/dataset_LHEE_RHEE_Z_PHASE_STD/"
CONTROL_PATH = os.path.join(SAVEPATH, "Controls")
PD_PATH = os.path.join(SAVEPATH, "PD")

if os.path.exists(SAVEPATH) == False: os.mkdir(SAVEPATH)
if os.path.exists(CONTROL_PATH) == False: os.mkdir(CONTROL_PATH)
if os.path.exists(PD_PATH) == False: os.mkdir(PD_PATH)

In [85]:
patients = getPatientsTable()
# patients.head()

In [86]:
CONTROL = patients[patients["Category"] == "Controls"]
PD = patients[patients["Category"] == "PD"]

In [87]:
print("Controls count:", len(CONTROL))
print("PD count:", len(PD))

Controls count: 22
PD count: 83


In [88]:
patient_initial = CONTROL.iloc[0]["Patient"]
print(patient_initial)

BHY


In [89]:
def getSTD(TARGET_CATE, patient_initial, filenamePostfix, idx):
    data = pd.read_csv(os.path.join(os.path.join(PHASEPATH, TARGET_CATE), f"PHASE_{patient_initial}_{filenamePostfix}{idx+1}.csv"))
    start = [np.where(data["PHASE"] == 1)[0][0]] + [ x for idx, x in enumerate(np.where(data["PHASE"] == 1)[0][1:]) if x -np.where(data["PHASE"] == 1)[0][idx] > 5 ]

    STD_IDX = 0

    for idx in start:
        if df.iloc[idx]["LHEE_Z"] < df.iloc[idx]["RHEE_Z"]:
#             print(data.iloc[idx][["LHEE_Z", "RHEE_Z", "LHEE_Z_DIFF", "RHEE_Z_DIFF"]])
            STD_IDX = idx
            break
        
    data = data.iloc[STD_IDX:]
    
    INIT_STD = data.iloc[0]["LHEE_X"], data.iloc[0]["LHEE_Y"], data.iloc[0]["LHEE_Z"]

    for col in data.columns.values[2:-3]:
        if col.endswith("X"):
            data[col] = data[col] - INIT_STD[0]
            after = data[col]

        if col.endswith("Y"):
            data[col] = data[col] - INIT_STD[1]
            after = data[col]    

        if col.endswith("Z"):
            data[col] = data[col] - INIT_STD[2]
            after = data[col]
    print(len(data), end='\n')
    data.to_csv(os.path.join(PATH, f"LHEE0_{patient_initial}_{filenamePostfix}{idx+1}.csv"), index=False)

    return STD_IDX

In [90]:
def applyStandard(dataFrame, TARGET_CATE, PATH):
    def getSTD(TARGET_CATE, patient_initial, filenamePostfix, idx:int):
        data = pd.read_csv(os.path.join(os.path.join(PHASEPATH, TARGET_CATE), f"PHASE_{patient_initial}_{filenamePostfix}{idx+1}.csv"))
        start = [np.where(data["PHASE"] == 1)[0][0]] + [ x for idx, x in enumerate(np.where(data["PHASE"] == 1)[0][1:]) if x -np.where(data["PHASE"] == 1)[0][idx] > 5 ]

        STD_IDX = 0

        for idx in start:
            if data.iloc[idx]["LHEE_Z"] < data.iloc[idx]["RHEE_Z"]:
    #             print(data.iloc[idx][["LHEE_Z", "RHEE_Z", "LHEE_Z_DIFF", "RHEE_Z_DIFF"]])
                STD_IDX = idx
                break
        print(len(data), STD_IDX, end='| ')
        data = data.iloc[STD_IDX:]

        INIT_STD = data.iloc[0]["LHEE_X"], data.iloc[0]["LHEE_Y"], data.iloc[0]["LHEE_Z"]

        for col in data.columns.values[2:-3]:
            if col.endswith("X"):
                data[col] = data[col] - INIT_STD[0]
                after = data[col]

            if col.endswith("Y"):
                data[col] = data[col] - INIT_STD[1]
                after = data[col]    

            if col.endswith("Z"):
                data[col] = data[col] - INIT_STD[2]
                after = data[col]
        print(len(data), end='\n')
        data.to_csv(os.path.join(PATH, f"LHEE0_{patient_initial}_{filenamePostfix}{idx+1}.csv"), index=False)

        return STD_IDX
    def applyStandardXYZ(dataList, PATH, TARGET_CATE, patient_initial, filenamePostfix):
        for idx, data in enumerate(dataList):
            # 필요에 따라 바꿔야함
            STD_IDX = getSTDidx(TARGET_CATE, patient_initial, filenamePostfix, idx)
            print(len(data), STD_IDX, end='| ')
            data = data.iloc[STD_IDX:]
            INIT_STD = data.iloc[0]["LHEE_X"], data.iloc[0]["LHEE_Y"], data.iloc[0]["LHEE_Z"]

            for col in data.columns.values[2:-3]:
                if col.endswith("X"):
                    data[col] = data[col] - INIT_STD[0]
                    after = data[col]

                if col.endswith("Y"):
                    data[col] = data[col] - INIT_STD[1]
                    after = data[col]    

                if col.endswith("Z"):
                    data[col] = data[col] - INIT_STD[2]
                    after = data[col]
            print(len(data), end='\n')
            data.to_csv(os.path.join(PATH, f"LHEE0_{patient_initial}_{filenamePostfix}{idx+1}.csv"), index=False)
            
    for patient_initial in dataFrame["Patient"]:
        print(patient_initial)
        BWdata, FWdata = getPatientData(TARGET_CATE, patient_initial)

#         applyStandardXYZ(BWdata, PATH, TARGET_CATE, patient_initial, "BW")
#         applyStandardXYZ(FWdata, PATH, TARGET_CATE, patient_initial, "FW")
        getSTD(TARGET_CATE, patient_initial, "FW", 0)
        getSTD(TARGET_CATE, patient_initial, "FW", 1)
        getSTD(TARGET_CATE, patient_initial, "FW", 2)

In [91]:
applyStandard(CONTROL, "Controls", CONTROL_PATH)

BHY
311 89| 222
316 79| 237
290 71| 219
CHH
329 83| 246
312 87| 225
322 83| 239
HSH
351 69| 282
356 34| 322
331 1| 330
JHY
269 12| 257
248 1| 247
282 12| 270
JJG
784 140| 644
793 96| 697
695 153| 542
JKJ
293 85| 208
253 58| 195
255 60| 195
KES
285 1| 284
317 1| 316
316 82| 234
KMS
936 46| 890
882 93| 789
816 191| 625
KangYK
277 66| 211
292 64| 228
328 47| 281
KimDS
382 10| 372
397 1| 396
360 34| 326
KimYC
298 61| 237
288 66| 222
343 89| 254
KimYJ
273 1| 272
258 4| 254
289 64| 225
LeeNS
385 1| 384
358 79| 279
371 21| 350
NTH
946 5| 941
829 147| 682
982 172| 810
ODS


FileNotFoundError: [Errno 2] No such file or directory: '/Users/sujinlee/desktop/git_repositories/nims_pd/dataset_LHEE_RHEE_Z_PHASE/Controls/PHASE_ODS_FW1.csv'

In [92]:
applyStandard(PD, "PD", PD_PATH)

AMJ
441 54| 387
355 1| 354
339 1| 338
BDY
312 26| 286
300 63| 237
284 23| 261
BGH


FileNotFoundError: [Errno 2] No such file or directory: '/Users/sujinlee/desktop/git_repositories/nims_pd/dataset_LHEE_RHEE_Z_PHASE/PD/PHASE_BGH_FW1.csv'

---

In [8]:
TARGET_CATE = "Controls"
TARGET_PATIENT_INITIAL = CONTROL.iloc[0]["Patient"]

print("Target Category:", TARGET_CATE)
print("Target Patient Initial:", TARGET_PATIENT_INITIAL)

Target Category: Controls
Target Patient Initial: BHY


In [33]:
BWdata, FWdata = getPatientData(TARGET_CATE, TARGET_PATIENT_INITIAL)

In [ ]:
BWdata[0].head()

,Frame,Sub Frame,LFHD_X,LFHD_Y,LFHD_Z,RFHD_X,RFHD_Y,RFHD_Z,LBHD_X,LBHD_Y,...,RHEE_Z,RTOE_X,RTOE_Y,RTOE_Z,CentreOfMass_X,CentreOfMass_Y,CentreOfMass_Z,CentreOfMassFloor_X,CentreOfMassFloor_Y,CentreOfMassFloor_Z
0,362,0,-67.340721,1411.976563,1462.628540,51.518406,1423.178467,1465.475586,-56.401962,1258.044556,...,42.530231,119.282578,1569.665527,41.352478,2.873885,1364.604126,843.360352,2.873885,1364.604126,0.0
1,363,0,-67.416672,1411.767822,1462.584351,51.419613,1422.962646,1465.438965,-56.486877,1257.829224,...,42.530090,119.282570,1569.661987,41.356304,2.813028,1364.414673,843.307312,2.813028,1364.414673,0.0
2,364,0,-67.775475,1410.832275,1462.389526,50.997658,1421.992432,1465.258057,-56.841885,1256.835205,...,42.527367,119.271065,1569.631714,41.368683,2.542822,1363.561768,843.068176,2.542822,1363.561768,0.0
3,365,0,-68.604591,1408.725220,1461.947876,50.092590,1419.817261,1464.827148,-57.603889,1254.555054,...,42.514172,119.240829,1569.552002,41.398766,1.934577,1361.649658,842.535645,1.934577,1361.649658,0.0
4,366,0,-69.962646,1405.297607,1461.229614,48.665070,1416.319824,1464.127686,-58.820278,1250.876953,...,42.476604,119.209480,1569.429810,41.470062,0.942147,1358.567017,841.683594,0.942147,1358.567017,0.0


In [35]:
INIT_STRN = BWdata[0].iloc[0]["STRN_X"], BWdata[0].iloc[0]["STRN_Y"], BWdata[0].iloc[0]["STRN_Z"]

In [36]:
for col in BWdata[0].columns.values[2:]:
#     print(col)
    if col.endswith("X"):
        BWdata[0][col] = BWdata[0][col] - INIT_STRN[0]
        after = BWdata[0][col]
        
    if col.endswith("Y"):
        BWdata[0][col] = BWdata[0][col] - INIT_STRN[1]
        after = BWdata[0][col]    
    
    if col.endswith("Z"):
        BWdata[0][col] = BWdata[0][col] - INIT_STRN[2]
        after = BWdata[0][col]

In [37]:
BWdata[0].head()

,Frame,Sub Frame,LFHD_X,LFHD_Y,LFHD_Z,RFHD_X,RFHD_Y,RFHD_Z,LBHD_X,LBHD_Y,...,RHEE_Z,RTOE_X,RTOE_Y,RTOE_Z,CentreOfMass_X,CentreOfMass_Y,CentreOfMass_Z,CentreOfMassFloor_X,CentreOfMassFloor_Y,CentreOfMassFloor_Z
0,362,0,-74.032479,-65.109741,470.366211,44.826648,-53.907837,473.213257,-63.093720,-219.041748,...,-949.732098,112.590820,92.579223,-950.909851,-3.817873,-112.482178,-148.901977,-3.817873,-112.482178,-992.262329
1,363,0,-74.108430,-65.318482,470.322022,44.727855,-54.123658,473.176636,-63.178635,-219.257080,...,-949.732239,112.590812,92.575683,-950.906025,-3.878730,-112.671631,-148.955017,-3.878730,-112.671631,-992.262329
2,364,0,-74.467233,-66.254029,470.127197,44.305900,-55.093872,472.995728,-63.533643,-220.251099,...,-949.734962,112.579307,92.545410,-950.893646,-4.148936,-113.524536,-149.194153,-4.148936,-113.524536,-992.262329
3,365,0,-75.296349,-68.361084,469.685547,43.400832,-57.269043,472.564819,-64.295647,-222.531250,...,-949.748157,112.549071,92.465698,-950.863563,-4.757181,-115.436646,-149.726684,-4.757181,-115.436646,-992.262329
4,366,0,-76.654404,-71.788697,468.967285,41.973312,-60.766480,471.865357,-65.512036,-226.209351,...,-949.785725,112.517722,92.343506,-950.792267,-5.749611,-118.519287,-150.578735,-5.749611,-118.519287,-992.262329
